In [1]:
# clone repository for deepsort with yolov4
# forked from theAIGuysCode/yolov4-deepsort
# !git clone https://github.com/zyerusha/yolov4-deepsort  

In [2]:
# uncomment below line ONLY if needed
!pip install -r requirements-gpu.txt

In [3]:
from shutil import copy2
src_dir = '../yolo/weights/yolov4.weights'
tgt_dir = './yolov4.weights'
copy2(src_dir, tgt_dir)

FileNotFoundError: [Errno 2] No such file or directory: '../yolo/weights/yolov4.weights'

In [ ]:
# Convert darknet weights to tensorflow model
%cd tensorflow_yolov4_tflite/
!python save_model.py --model yolov4 --weights ../yolov4.weights --output ../checkpoints/yolov4
%cd ..

In [ ]:
import shutil

def CopyNeededFolders(folder_name, src_dir, target_dir = '.'):
    tgt_name = os.path.join(target_dir, folder_name)
    src_name = os.path.join(src_dir, folder_name)
    if os.path.isdir(tgt_name): 
        shutil.rmtree(tgt_name) 
    shutil.copytree(src_name,tgt_name)
    print(f'Copied folder {src_name} --> {tgt_name}')

CopyNeededFolders('data', './tensorflow_yolov4_tflite', '.') 
CopyNeededFolders('core', './tensorflow_yolov4_tflite', '.') 
CopyNeededFolders('model_data', '../yolov4-deepsort', '.') 


In [ ]:
# setup
dataset_dir_path = '../datasets/VIRAT/'
image_ext = '.jpg'
video_max_frames = 2000

#video
video_ext = '.mp4'
video_name = 'VIRAT_S_050000_07_001014_001126'
video_name_orig = video_name + video_ext
video_dest_path = './processed/' +  video_name + '/'
video_src_path = dataset_dir_path + 'Videos/Ground/'

using_yml = True

# annotations
saved_csv = video_dest_path + 'df_bbox.csv'


yml_video_name = 'gt'
yolo_video_name = 'yolo'

# annotations_path = dataset_dir_path + 'viratannotations/train/' + video_name +'/'
annotations_path = dataset_dir_path + 'viratannotations/validate/' + video_name +'/'
ann_activities_file = annotations_path + video_name + '.activities.yml'
ann_geom_file = annotations_path + video_name + '.geom.yml'
ann_regions_file = annotations_path + video_name + '.regions.yml'
ann_types_file = annotations_path + video_name + '.types.yml'


src_video = os.path.join(video_src_path, video_name_orig)
gt_video_name = yml_video_name + '_' # + '.mp4'#video_ext
yolo_video_name = yolo_video_name + '_'# + '.mp4'#video_ext

In [ ]:
# Create directory to store new video
if not os.path.exists(video_dest_path):
    os.makedirs(video_dest_path)

if not os.path.exists(saved_csv):
    with open(ann_types_file) as yaml_file:
        yaml_contents = load(yaml_file, Loader=SafeLoader)
    yaml_df = json_normalize(yaml_contents)
    yaml_df
    for col in yaml_df.columns:
        type_name = col.split('.')[-1]
        if not (type_name == 'id1'):
            yaml_df.loc[yaml_df[col] == 1, col] = type_name
    
    yaml_df = yaml_df[yaml_df['types.id1'].notna()].reset_index().dropna(axis=1, how='all')  
    type_df = yaml_df.ffill(axis=1).iloc[:,-1].to_frame(name='category')
    type_df.insert(0, "id", yaml_df['types.id1'])
    type_df

In [ ]:
import pandas as pd

# using annotations:
print("Loading annotations...")
def add_category_type(row):
  id = row['object_id']
  val = type_df.loc[type_df['id'] == id, 'category'].iloc[0]
  return val


if os.path.exists(saved_csv):
  df_bbox = pd.read_csv(saved_csv)
else:
  with open(ann_geom_file) as yaml_file:
      yaml_contents = load(yaml_file, Loader=SafeLoader)
  yaml_df = json_normalize(yaml_contents)

  df_bbox = yaml_df[['geom.id1','geom.ts0','geom.ts1','geom.g0']].dropna().reset_index()
  df_bbox.rename(columns={'geom.id1': 'object_id', 'geom.ts0': 'frame_id','geom.ts1': 'time_sec', 'geom.g0': 'bbox'}, inplace=True)
  df_bbox['bbox'] = df_bbox['bbox'].str.split()
  df_tmp = pd.DataFrame(df_bbox['bbox'].to_list(), columns = ['bb_left', 'bb_top', 'bb_right', 'bb_bottom'])
  df_bbox = pd.concat([df_bbox, df_tmp], axis=1).drop(columns=['bbox'])

  df_bbox['category'] = df_bbox.apply(lambda row: add_category_type(row), axis=1) 
  df_bbox.drop(columns=['index'], axis=1, inplace=True)
  # df_bbox.set_index['index'] 
  df_bbox.to_csv(saved_csv, index = False)
    

df_bbox.head()

In [ ]:
# import cv2
# from utils.velocity_utils import VelocityUtils
# velUtils = VelocityUtils() 

# vidcap = cv2.VideoCapture(src_video)
# fps = 30
# scale = 1/15
# if vidcap.isOpened():
#     fps = vidcap.get(cv2.CAP_PROP_FPS)  
#     print(fps)

# df_bbox = velUtils.add_vel_to_df(df_bbox, fps, scale)

# df_bbox.head()

In [ ]:
import os
import cv2
from utils.yolo_utils import YoloUtils
from utils.video_utils import VideoUtils
from utils.folder_utils import FolderUtils
from utils.deepsort_yolo import DeepsortYolo
yUtils = YoloUtils()
vUtils = VideoUtils() 
deepsortYolo  = DeepsortYolo()

# yolo_weight_file = "../yolo/weights/yolov3.weights"
# yolo_cfg_file = "../yolo/darknet/cfg/yolov3.cfg"
yolo_names_file = "../yolo/darknet/data/coco.names"
# yolo_weight_file = "yolov4.weights"
# yolo_cfg_file = "../yolo/darknet/cfg/yolov3.cfg"
# yolo_names_file = "../yolo/darknet/data/coco.names"
start_time = 0
video_duration=10#None
video_in = cv2.VideoCapture(src_video)
if video_in.isOpened():
    fps, total_frames, frame_size = vUtils.GetVideoData(video_in)
    start_count, end_count = vUtils.GetStartEndCount(fps, total_frames, start_time, video_duration)
    video_duration = int(end_count / fps)
    video_in.release()


yolo_filename = os.path.join(video_dest_path, VideoUtils.AddTimestampToName(yolo_video_name, start_time, video_duration))
gt_filename = os.path.join(video_dest_path, VideoUtils.AddTimestampToName(gt_video_name, start_time, video_duration))

tracker_file_csv = yolo_filename + '.csv'
tracker_file_mp4 = yolo_filename + '.mp4'
gt_file_mp4 = gt_filename + '.mp4'
if not os.path.exists(tracker_file_csv):
    tracker_video_out, trk_bbox = deepsortYolo.ProcessVideo("./checkpoints/yolov4", yolo_names_file, src_video, video_dest_path, tracker_file_mp4, start_time_sec=start_time, duration_sec=video_duration, save_images=False)
    trk_bbox.to_csv(tracker_file_csv, index=False)

    if os.path.exists(gt_filename):
        df_bbox_filtered = df_bbox[df_bbox['category'] == 'Vehicle']
        gt_video_out, bbox_gt = vUtils.AnnotateVideo(video_dest_path, tracker_video_out, gt_file_mp4, df_bbox_filtered, start_time, video_duration)
    


# 
# yolo_video_out, trk_bbox = deepsortYolo.ProcessVideo("./checkpoints/yolov4", yolo_names_file, src_video, video_dest_path, yolo_filename + '.mp4', start_time_sec=start_time, duration_sec=video_duration, save_images=False)
# trk_bbox.to_csv(os.path.join(video_dest_path,'image_data.csv'), index=False)
trk_bbox = pd.read_csv(tracker_file_csv)
# gt_video_out, bbox_gt = vUtils.AnnotateVideo(video_dest_path, yolo_video_out, gt_filename + '.mp4', df_bbox_vehicle, start_time, video_duration)
trk_bbox

In [ ]:
import numpy as np
from utils.video_utils import VideoUtils
from utils.velocity_utils import VelocityUtils
from utils.filtering_utils import Filters
vUtils = VideoUtils() 
velUtils = VelocityUtils()


df = pd.DataFrame(data=trk_bbox)
df.head(50)
  
scale = 1/15

df_person = df[(df['object_id'] == 4)]
human_height_avg = 1.75
human_pixel_height =(df_person['bb_bottom'].mean() - df_person['bb_top'].mean()) 
scale = human_height_avg / human_pixel_height
print(f'human pixel high: {human_pixel_height}, avg human height: {human_height_avg}, scale: {scale}')


df = df.reset_index()
df = df.drop(columns = ['index'])
for id in df['object_id'].unique():
  # this adds velocity to the dataframe
  df = velUtils.AddVelocity(df, id, fps, scale, ['bb_left', 'bb_top', 'bb_right', 'bb_bottom'])
  # df = velUtils.FilterVelocity(df, id, fps)

# df.to_csv(os.path.join(video_dest_path,'image_data_vel.csv'), index=False)

# df['vel'][df['vel'] < 1] = 0

df_test = pd.DataFrame(df[(df['object_id'] == 13)])

# df_test['vel_filt'] =  Filters.ButterLowpass(df_test['vel'], 2, fps, 1)


df_test.head(50)


# 

In [ ]:
# import matplotlib.pyplot as plt
# df['vel_filt'] =  ButterLowpassFilter(df['vel'], float(1/fps), fps, 1)
# tmp= pd.DataFrame([])
# tmp['x'] = df[(df['object_id'] == 5)]['Frame']
# tmp['y1'] = df[(df['object_id'] == 5)]['vel']
# tmp['y2'] = df[(df['object_id'] == 5)]['vel_filt']
# tmp.head()
# plt.plot(tmp['x'],tmp['y1'], tmp['x'],tmp['y2'])
# plt.show()

In [ ]:
from pathlib import Path

def AddVelocityToFrame(frame, df_info):
    for index, row in df_info.iterrows():
        # bb_gt = row['gt bbox']
        # txt_val = "{vel:.2f}".format(vel = row['vel_filt'])
        txt_val = "{vel:.2f}".format(vel = row['vel'])
        cv2.putText(frame, txt_val, (int(row['x']), int(row['y'])), cv2.FONT_HERSHEY_COMPLEX, 0.75, (0, 255, 255), 1)
        # img = vUtils.PrintText(img, txt_val, row['x'], row['y'], -10, 10, cv2.FONT_HERSHEY_COMPLEX, 0.75, (255, 255, 255))
    return frame


def AddVelocityToVideo(orig_v_full_path, df):
    if (not orig_v_full_path):
        raise Exception(f"File not found: {orig_v_full_path}")

    p = Path(orig_v_full_path)
    filename = os.path.join(p.parent, "vel_" + p.name)

    bbox_data = {}
    # Open original video
    video_in = cv2.VideoCapture(orig_v_full_path)
    if video_in.isOpened():
        fps, total_frames, frame_size = vUtils.GetVideoData(video_in)
        count = 0
        # setting CV_CAP_PROP_POS_FRAMES at count
        video_in.set(cv2.CAP_PROP_POS_FRAMES, count)
        
        fourcc = cv2.VideoWriter_fourcc(*"XVID")
        video_out = cv2.VideoWriter(
            filename, fourcc, int(fps), frame_size)

        i = 0
        while (True):
            success, frame = video_in.read()
            if(success):
                # height, width, layers = frame.shape
                # image_size = (width, height)

                # All bounding box info for this frame
                # Add all annotations to the frame
                frame = AddVelocityToFrame(frame, df[df['Frame'] == count])
                i += 1
                video_out.write(frame)
                count += 1

            else:
                break

        video_in.release()  # done with original video
        video_out.release()

        print("Done: Created video: " + filename)

    cv2.destroyAllWindows()

# yolo_video_out = os.path.join(video_dest_path,'0-10_yolo_VIRAT_S_050000_07_001014_001126.mp4')

AddVelocityToVideo(tracker_file_mp4, df)

In [ ]:
from utils.bbox_utils import Bbox
import pandas as pd

def GetMaxCorrelation(bb_gt, pred_bboxes, frame):
    max_iou = 0.0
    max_info = [frame, bb_gt, bb_gt, max_iou]
    found = False

    for bb_pred in pred_bboxes:
        gt = Bbox(bb_gt[0],bb_gt[1],bb_gt[2],bb_gt[3])
        pred = Bbox(bb_pred[0],bb_pred[1],bb_pred[2],bb_pred[3])
        iou = gt.IOU(pred)

        if (max_iou < iou) & (0.5 < iou):
            found = True
            max_iou = iou
            max_info = [frame, bb_gt, bb_pred, iou]

    return found, max_info

df = pd.DataFrame(columns=['frame idx', 'gt bbox', 'pred bbox', 'iou'])
print(f'# of frames in video: {len(bbox_gt)}')
# for i_frame in range(1):#range(len(bbox_gt)): #scan all frames in video
for i_frame in range(len(bbox_gt)): #scan all frames in video
    data = []
    for bb_gt in bbox_gt[i_frame]:
        found, info = GetMaxCorrelation(bb_gt, bbox_yolo[i_frame], i_frame)
        if(found):
            data.append(info)
    
    df_frame = pd.DataFrame(data, columns=['frame idx', 'gt bbox', 'pred bbox', 'iou'])
    df = pd.concat([df, df_frame], ignore_index=True)



mPA = df['iou'].sum()/len(df)
print(mPA)
df.head(30)


In [ ]:
# add IOU to video
from video_utils_virat import VideoUtils
vUtils = VideoUtils() 

gt_video_out

def AddIouToFrame(img, df_info):
    for index, row in df_info.iterrows():
        bb_gt = row['gt bbox']
        txt_val = "{iou:.4f}".format(iou = row['iou'])
        bb = Bbox(bb_gt[0],bb_gt[1],bb_gt[2],bb_gt[3])
        img = vUtils.PrintText(img, txt_val, bb.center_x, bb.center_y, -10, 10, cv2.FONT_HERSHEY_COMPLEX, 0.75, (255, 255, 255))
    return img


def AddIouToVideo(output_dir, orig_v_full_path, df_info):
    if (not orig_v_full_path):
        raise Exception(f"File not found: {orig_v_full_path}")

    bbox_data = {}
    # Open original video
    video_in = cv2.VideoCapture(orig_v_full_path)
    if video_in.isOpened():
        fps, total_frames, frame_size = vUtils.GetVideoData(video_in)
        count = 0
        # setting CV_CAP_PROP_POS_FRAMES at count
        video_in.set(cv2.CAP_PROP_POS_FRAMES, count)
        full_filename = os.path.join(output_dir,'IOU.mp4')
        if os.path.exists(full_filename):
            os.remove(full_filename)

        fourcc = cv2.VideoWriter_fourcc(*"XVID")
        video_out = cv2.VideoWriter(
            full_filename, fourcc, int(fps), frame_size)

        i = 0
        while (True):
            success, img = video_in.read()
            if(success):
                height, width, layers = img.shape
                image_size = (width, height)

                # All bounding box info for this frame
                # Add all annotations to the frame
                df_img = df_info[df_info['frame idx'] == count]
                img = AddIouToFrame(img, df_img)
                i += 1
                video_out.write(img)
                count += 1

            else:
                break

        video_in.release()  # done with original video
        video_out.release()

        print("Done: Created video: " + full_filename)

    cv2.destroyAllWindows()


AddIouToVideo(video_dest_path, gt_video_out, df)


In [ ]:
# from video_utils_virat import YoloUtils
# yUtils = YoloUtils()
# yolo_weight_file = "../yolo/weights/yolov3.weights"
# yolo_cfg_file = "../yolo/darknet/cfg/yolov3.cfg"
# yolo_names_file = "../yolo/darknet/data/coco.names"
# yUtils.YoloOnScreen(yolo_weight_file, yolo_cfg_file, yolo_names_file, 0,0,800,600)
